In [55]:
from scipy.fftpack import dct
import cv2 as cv2

In [56]:
compression_level=0.01#1 means no compression, the smaller, the more compression. Must be greater than 0

In [57]:
img1= cv2.imread('test.png', cv2.COLOR_BGR2RGB)#reading in the image. Please change the file path here if you wish to read another image

imgYUV=cv2.cvtColor(img1,cv2.COLOR_BGR2YUV)

#store size for later use
size0=img1.shape[0]
size1=img1.shape[1]

In [58]:
def dct2(a):#a simple helper function for dct2
    return dct(dct(a.T, norm='ortho').T, norm='ortho')


In [59]:
#this function is used to encode a channel of the image using dct
def chanel_compression(image, compression_level_function):
    img_dct=dct2(image)
    img_dct[compression_level_function:, compression_level_function:]=0
    return img_dct
    

In [60]:
encoded_y=dct2(imgYUV[:, :, 0].copy())
encoded_u=chanel_compression(imgYUV[:, :, 1].copy(), int(img_gray.shape[0] * compression_level))
encoded_v=chanel_compression(imgYUV[:, :, 2].copy(), int(img_gray.shape[0] * compression_level))


# huffman encoding
for this to work, please install the package dahuffman. pip install huffman

In [61]:
from dahuffman import HuffmanCodec

#encode y
print("huffman encoding running for layer y")
list_y=list(encoded_y.flatten())#convert the 2d-image layer to a list. This is necessaryas huffman requires a list as input
code_y=HuffmanCodec.from_data(list_y)#create the code book
h_encoded_y=code_y.encode(list_y)#encode using the generated code book

#encode u
print("huffman encoding running for layer u")
list_u=list(encoded_u.flatten())#convert the 2d-image layer to a list. This is necessaryas huffman requires a list as input
code_u=HuffmanCodec.from_data(list_u)#create the code book
h_encoded_u=code_u.encode(list_u)#encode using the generated code book

#encode v
print("huffman encoding running for layer v")
list_v=list(encoded_v.flatten())#convert the 2d-image layer to a list. This is necessaryas huffman requires a list as input
code_v=HuffmanCodec.from_data(list_v)#create the code book
h_encoded_v=code_v.encode(list_v)#encode using the generated code book


huffman encoding running for layer y
huffman encoding running for layer u
huffman encoding running for layer v


# store everything to disk

In [62]:
class CompressedImage:
    def __init__(self,size0, size1, code_y,code_u,code_v,h_encoded_y,h_encoded_u,h_encoded_v ):
        self.size0=size0
        self.size1=size1
        
        self.code_y=code_y
        self.code_u=code_u
        self.code_v=code_v
        
        self.h_encoded_y=h_encoded_y
        self.h_encoded_u=h_encoded_u
        self.h_encoded_v=h_encoded_v
    

In [63]:
compressed_image=CompressedImage(size0, size1, code_y,code_u,code_v,h_encoded_y,h_encoded_u,h_encoded_v )

In [64]:
#for optimal performance, use cPickle if available. If not, use pickle
try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle

with open("test.bin", 'wb') as outp:  
    pickle.dump(compressed_image, outp, pickle.HIGHEST_PROTOCOL)
